In [1]:
from training_utils import load_model_data 
from training_utils import LinePlot
from lens import get_lens_loss
import torch
from tqdm import tqdm
import numpy as np
import pickle
import math


## Load model and data
We will be using gpt2 small for now.

In [2]:
model_name = "gpt2-small"
batch_size = 32
device, model, tokenizer, owt_iter = load_model_data(model_name, batch_size)
n_layers = model.cfg.n_layers
n_heads = model.cfg.n_heads
head_dim = model.cfg.d_head
d_model = model.cfg.d_model
lr = 1e-2

folder = "./results/"
shared_bias = False

print(n_layers, n_heads, head_dim, d_model)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Loaded pretrained model gpt2-small into HookedTransformer


Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/100000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (61422 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (58948 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (59558 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (71857 > 1024). Running this sequence through the model will result in indexing errors


12 12 64 768


## Create Modal Lens Prameters

In [3]:
prior_bias = [model.blocks[i].attn.b_O.clone() for i in range(n_layers)]

attn_bias = [
    torch.nn.Parameter(
        (prior_bias[i] if shared_bias else prior_bias[i].repeat(i + 1, 1)).to(device)
    )
    for i in range(n_layers)
]

lens_optimizer = torch.optim.AdamW(attn_bias, lr=lr, weight_decay=0)

for param in model.parameters():
    param.requires_grad = False


## Training Loop

In [4]:
lp = LinePlot([*[f"kl_loss_{k}" for k in range(n_layers)], "step_size"])

for i in tqdm(range(5000)):
    batch = next(owt_iter)["tokens"]
    lens_optimizer.zero_grad()

    kl_losses, _ = get_lens_loss(batch, model, n_layers, attn_bias,batch_size)
    kl_losses = kl_losses.mean(dim=-1)
    loss = torch.clamp(kl_losses.sum(), max=100)
    loss.backward()

    prev_weights = torch.cat(attn_bias, dim=0).detach()

    lens_optimizer.step()

    step_sz = (torch.cat(attn_bias, dim=0) - prev_weights).abs().sum()
    lp.add_entry(
        {
            "step_size": step_sz.item(),
            **{f"kl_loss_{k}": kl_losses[k].item() for k in range(n_layers)},
        }
    )

    # lens_scheduler.step()

    if math.isnan(lp.stat_book["step_size"][-1]):
        break

    if i % 250 == 0:
        lp.plot(subplots=3, save=f"{folder}/train.png", twinx=False, mv=20)
        with open(f"{folder}/modal_lens_weights.pkl", "wb") as f:
            pickle.dump(attn_bias, f)

  0%|          | 0/5000 [00:00<?, ?it/s]


TypeError: get_lens_loss() missing 4 required positional arguments: 'model', 'n_layers', 'attn_bias', and 'batch_size'